# Install required Packages

In [ ]:
!pip install transformers --quiet
!pip install langchain_huggingface --quiet
!pip install langchain --quiet
!pip install gradio --quiet
!pip install bitsandbytes --quiet
!pip install torch torchvision torchaudio --quiet
!pip install huggingface_hub --quiet
!pip install huggingface_hub[hf_xet] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Importing required libraries

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import gradio as gr
import os
import warnings
warnings.filterwarnings('ignore')

# Importing HF Token

In [ ]:
from google.colab import userdata
huggingface_user=userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import whoami
print(whoami(token=huggingface_user))

{'type': 'user', 'id': '66957888f4d5f5d06ca36462', 'name': 'Suman2004', 'fullname': 'Suman Roy', 'email': 'sumanroy202400@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': 1748735999, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/66957888f4d5f5d06ca36462/BQzGRCRMfy4WL6NSsAICJ.jpeg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'sent_token', 'role': 'write', 'createdAt': '2025-05-14T17:59:22.556Z'}}}


# Loading the finetuned model

In [ ]:
from huggingface_hub import login
login(token=huggingface_user)
model_id = "Suman2004/lang-trans-sentiment-analyser__finetuned-llama-3.2-3b-instruct-unsloth-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10000
)
hf = HuggingFacePipeline(pipeline=pipe)

# Launching the gradio app

In [ ]:
system_message =''' You are an advanced language model specializing in comprehensive sentiment and language analysis. Your task is to analyze the input text and provide two distinct outputs:
  1. Sentiment Analysis:
               Identify the sentiment expressed in the text. Categorize the sentiment using the following specific emotion labels:
                                               Joy/Happiness
                                               Sadness
                                               Anger
                                               Fear
                                               Surprise
                                               Disgust
                                               Frustration
                                               Excitement
                                               Anxiety
                                               Hope
               If the text expresses a mix of emotions, identify the most prominent one.
               If the text is neutral or objective, state Sentiment-Neutral.
  2. Language Analysis:
               Identify the current language of the input and provide translated text into English and original full name of the language of the input.'''

In [ ]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Q: Du ist sehr klug und nett !!"},]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 30000,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

1. Sentiment Analysis:
   - Detected Sentiment: anger

2. Language Analysis:
   - Original Language: German
   - English Translation: Q: You are very clever and nice!!<|eot_id|>


In [ ]:
import gradio as gr

def analyze_text(input_text):
    system_message = '''You are an advanced language model specializing in comprehensive sentiment and language analysis. Your task is to analyze the input text and provide two distinct outputs:
1. Sentiment Analysis:
   Identify the sentiment expressed in the text. Categorize the sentiment using the following specific emotion labels:
       Joy/Happiness
       Sadness
       Anger
       Fear
       Surprise
       Disgust
       Frustration
       Excitement
       Anxiety
       Hope
   If the text expresses a mix of emotions, identify the most prominent one.
   If the text is neutral or objective, state Sentiment-Neutral.

2. Language Analysis:
   Identify the current language of the input and provide translated text into English and original full name of the language of the input.'''

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": input_text},
    ]

    # Apply the chat template
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    # Generate the response
    response = hf.invoke(formatted_prompt)
    assistant_response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1].split("<|eot_id|>")[0].strip()
    return assistant_response

# Gradio interface with an example
iface = gr.Interface(
    fn=analyze_text,
    inputs=gr.Textbox(lines=4, placeholder="Enter text here..."),
    outputs="text",
    title="Multilingual Sentiment and Language Analyzer",
    description="Enter text in any language. The model will detect the language, translate it to English, and analyze the sentiment.",
    examples=[
        ["Du ist sehr klug und nett !!"]
    ]
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b419efd00e34cc8889.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
